In [1]:
# import packages 
import pandas as pd 
import numpy as np 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error
from featurewiz import featurewiz
import seaborn as sns
import warnings
warnings.simplefilter("ignore")
np.random.seed(1234)

Imported featurewiz: advanced feature engg and selection library. Version=0.0.42
output = featurewiz(dataname, target, corr_limit=0.70,
                    verbose=2, sep=',', header=0, test_data='',
                    feature_engg='', category_encoders='')
Create new features via 'feature_engg' flag : ['interactions','groupby','target']
                                


In [2]:
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")
submission = pd.read_csv("SampleSubmission.csv")

In [3]:
data = pd.concat([train,test],sort=False).reset_index(drop=True)

In [4]:
data.shape

(28648, 19)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28648 entries, 0 to 28647
Data columns (total 19 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   ID                                       28648 non-null  object 
 1   country                                  28648 non-null  object 
 2   year                                     28648 non-null  int64  
 3   urban_or_rural                           28648 non-null  object 
 4   ghsl_water_surface                       28648 non-null  float64
 5   ghsl_built_pre_1975                      28648 non-null  float64
 6   ghsl_built_1975_to_1990                  28648 non-null  float64
 7   ghsl_built_1990_to_2000                  28648 non-null  float64
 8   ghsl_built_2000_to_2014                  28648 non-null  float64
 9   ghsl_not_built_up                        28648 non-null  float64
 10  ghsl_pop_density                         28648

In [6]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
year,28648.0,2010.271537,4.553145,1994.000000,2008.000000,2011.000000,2014.000000,2016.000000
ghsl_water_surface,28648.0,0.027018,0.095199,0.000000,0.000000,0.000000,0.000000,0.991100
ghsl_built_pre_1975,28648.0,0.041010,0.118721,0.000000,0.000000,0.000160,0.008011,0.894608
ghsl_built_1975_to_1990,28648.0,0.027820,0.072850,0.000000,0.000000,0.000602,0.010037,0.685010
ghsl_built_1990_to_2000,28648.0,0.013713,0.034208,0.000000,0.000037,0.000936,0.008823,0.515534
ghsl_built_2000_to_2014,28648.0,0.018369,0.040241,0.000000,0.000109,0.001727,0.015209,0.649159
ghsl_not_built_up,28648.0,0.872069,0.242707,0.000859,0.895615,0.992725,0.999592,1.000000
ghsl_pop_density,28648.0,95.101508,210.079701,0.000000,2.943082,16.424158,64.364772,1902.876955
landcover_crops_fraction,28648.0,21.097237,16.309999,0.000000,6.105505,19.297194,33.199903,80.064918
landcover_urban_fraction,28648.0,14.079153,23.917510,0.000000,0.705307,2.599028,13.063126,98.784092


In [7]:
data.isnull().any(axis=1)

0        False
1        False
2        False
3        False
4        False
         ...  
28643     True
28644     True
28645     True
28646     True
28647     True
Length: 28648, dtype: bool

In [8]:
data.fillna(method="ffill", limit=1, inplace=True)

In [9]:
data.isnull().any(axis=1)

0        False
1        False
2        False
3        False
4        False
         ...  
28643     True
28644     True
28645     True
28646     True
28647     True
Length: 28648, dtype: bool

In [10]:
#import preprocessing module 

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['urban_or_rural'] = le.fit_transform(data['urban_or_rural'])
data['country'] = le.fit_transform(data['country'])

In [11]:
# convert float dtypes to int
data["ghsl_water_surface"] = data['ghsl_water_surface'].apply(np.int64)
data["ghsl_built_pre_1975"] = data['ghsl_built_pre_1975'].apply(np.int64)
data["ghsl_built_1990_to_2000"] = data["ghsl_built_1990_to_2000"].apply(np.int64)
data["ghsl_built_2000_to_2014"] = data["ghsl_built_2000_to_2014"].apply(np.int64)
data["ghsl_not_built_up"] = data["ghsl_not_built_up"].apply(np.int64)
data["ghsl_pop_density"] = data["ghsl_pop_density"].apply(np.int64) 
data["landcover_crops_fraction"] = data["landcover_crops_fraction"].apply(np.int64)
data["landcover_urban_fraction"] = data["landcover_urban_fraction"].apply(np.int64)
data["landcover_water_permanent_10km_fraction"] = data["landcover_water_permanent_10km_fraction"].apply(np.int64)
data["landcover_water_seasonal_10km_fraction"] = data["landcover_water_seasonal_10km_fraction"].apply(np.int64)
data["nighttime_lights"] = data["nighttime_lights"].apply(np.int64) 
data["dist_to_shoreline"] = data["dist_to_shoreline"].apply(np.int64)
data["ghsl_built_1975_to_1990"] = data["ghsl_built_1975_to_1990"].apply(np.int64)
data["dist_to_capital"] = data["dist_to_capital"].apply(np.int64)


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28648 entries, 0 to 28647
Data columns (total 19 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   ID                                       28648 non-null  object 
 1   country                                  28648 non-null  int64  
 2   year                                     28648 non-null  int64  
 3   urban_or_rural                           28648 non-null  int64  
 4   ghsl_water_surface                       28648 non-null  int64  
 5   ghsl_built_pre_1975                      28648 non-null  int64  
 6   ghsl_built_1975_to_1990                  28648 non-null  int64  
 7   ghsl_built_1990_to_2000                  28648 non-null  int64  
 8   ghsl_built_2000_to_2014                  28648 non-null  int64  
 9   ghsl_not_built_up                        28648 non-null  int64  
 10  ghsl_pop_density                         28648

In [19]:
np.isnan(data.any(axis=1))

0        False
1        False
2        False
3        False
4        False
         ...  
28643    False
28644    False
28645    False
28646    False
28647    False
Length: 28648, dtype: bool

#### Split the data into independent features and target.

In [13]:
X = data.drop(["ID",'Target'],axis=1)
 
y = data.Target.values 

In [14]:
##Standardize the features by using StandardScaler from scikit-learn.
from sklearn.preprocessing import StandardScaler
X_scaled =  StandardScaler().fit_transform(X) 

In [15]:
X_train, X_valid, y_train, y_valid = train_test_split(X_scaled,y,test_size = 0.2, random_state=1)

In [21]:
X_train.shape, y_train.shape

((22918, 17), (22918,))

In [16]:
classifier = RandomForestRegressor()
 
classifier.fit(X_train,y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [22]:
np.where(np.isnan(X_train))

(array([], dtype=int64), array([], dtype=int64))

In [23]:
np.where(np.isnan(y_train))

(array([    4,     5,     6, ..., 22901, 22902, 22903]),)

In [25]:
data.head()

,ID,country,year,urban_or_rural,ghsl_water_surface,ghsl_built_pre_1975,ghsl_built_1975_to_1990,ghsl_built_1990_to_2000,ghsl_built_2000_to_2014,ghsl_not_built_up,ghsl_pop_density,landcover_crops_fraction,landcover_urban_fraction,landcover_water_permanent_10km_fraction,landcover_water_seasonal_10km_fraction,nighttime_lights,dist_to_capital,dist_to_shoreline,Target
0,ID_AAIethGy,7,2016,0,0,0,0,0,0,0,12,25,0,0,0,0,278,769,0.132783
1,ID_AAYiaCeL,7,2005,0,0,0,0,0,0,0,113,64,0,0,0,0,200,337,0.004898
2,ID_AAdurmKj,14,2009,0,0,0,0,0,0,1,0,4,0,0,0,0,642,169,0.097320
3,ID_AAgNHles,12,2015,0,0,0,0,0,0,0,5,25,2,11,0,0,365,613,0.304107
4,ID_AAishfND,9,2012,1,0,0,0,0,0,0,31,5,22,0,0,1,222,192,0.605328
